In [ ]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [ ]:
base_url = 'https://teamtreehouse.com/tracks/beginning-python'

In [ ]:
resp = requests.get(base_url, allow_redirects=False)
resp

In [ ]:
resp.content[:300]

In [ ]:
soup = bs(resp.content, 'html.parser')

In [ ]:
soup.find('p')

In [ ]:
all_ps = soup.find_all('p')
len(all_ps)

In [ ]:
all_ps
#all_ps[0]

In [ ]:
cards = soup.find_all('a', class_='card-box')

In [ ]:
len(cards)

In [ ]:
cards[0]

In [ ]:
def get_duration(card):
    duration_raw = card.find('span', class_='card-estimate').text
    time, units = duration_raw.split()
    assert(units in ['min', 'hours']), f'Unexpected time units: {units}'
    time = int(time)
    if units == 'hours':
        time *= 60
    return time

In [ ]:
def get_lessons(course_url):
    resp = requests.get(course_url, allow_redirects=False)
    soup = bs(resp.content, 'html.parser')
    units = soup.find_all('div', class_='featurette')
    lessons = []
    for unit in units:
        lesson_title = unit.find_all('h2')[0].text.strip()
        lessons.append(lesson_title)
    return lessons
    
    
get_lessons(
    'https://teamtreehouse.com/library/python-basics-3/stages')

In [ ]:
def get_data_from_card(card):
    title = card.find('h3', class_='card-title').text.strip()
    duration_in_minutes = get_duration(card) 
    description = card.find('p', class_='card-description').text.strip()
    course_url = (
        'https://teamtreehouse.com' + card.get('href') + '/stages')
    lessons = get_lessons(course_url)
    out = {'title': title,
           'duration_in_minutes': duration_in_minutes,
           'description': description,
           'lessons': lessons}
    return out
    
    
    
get_data_from_card(cards[0])

In [ ]:
all_data = [get_data_from_card(card) for card in cards]
all_data

In [ ]:
def json_to_dataframe(all_data):
    columns = all_data[0].keys()
    data = []
    for datum in all_data:
        row = [datum[col] for col in columns]
        data.append(row)
    df = pd.DataFrame(data=data, columns=columns)
    return df
    
df = json_to_dataframe(all_data)
df

In [ ]:
df.to_csv('python_listings.csv', index=False)